# Automatización de consultas a APIs
En este ejercicio haremos un tutorial en el que se desglosarán los pasos para crear **dos scripts independientes en Python** con el propósito de automatizar las tareas de recopilación y procesamiento proporcionados en el ejercicio anterior. Estos scripts estarán diseñados para ser programdos y ejecutados diariamente.

---

## **Script 1: Recopilación y Procesamiento de Datos de Open-Meteo**
El objetivo de este script es recopilar datos históricos de Open-Meteo, procesarlos y guardarlos como archivos CSV para su posterior análisis.

### **Código del Script**

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import quote

# Configuración
LAT, LON = 28.4161, -16.5446
DAYS = 7  # Últimos 7 días
END_DATE = datetime.now().date() - timedelta(days=0)
START_DATE = END_DATE - timedelta(days=DAYS)

# Función para obtener datos de Open-Meteo
def obtener_datos_historicos_open_meteo(start_date, end_date, lat, lon, variables="temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m"):
    url = "https://archive-api.open-meteo.com/v1/archive"
    response = requests.get(
        url,
        params={
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": end_date.strftime("%Y-%m-%d"),
            "hourly": variables
        }
    )
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Error al obtener datos de Open-Meteo")

def procesar_datos_open_meteo(data, id, agregar_diario=False):
    # Convertir el JSON a DataFrame
    df = pd.DataFrame(data['hourly'])
    df['time'] = pd.to_datetime(df['time'])
    df.rename(columns={"time": "Fecha"}, inplace=True)
    df.set_index('Fecha', inplace=True)
    df.dropna(inplace=True)
    # Agregar a frecuencia diaria
    if agregar_diario:
        df['Dia'] = df.index.date
        daily_df = df.groupby('Dia').agg(
            temperature = ('temperature_2m', 'mean'),  # Promedio de temperatura
            temperature_min = ('temperature_2m', 'min'),  # Minimo de temperatura
            temperature_max = ('temperature_2m', 'max'),  # Máximo de temperatura
            humidity =('relative_humidity_2m', 'mean'),  # Promedio de humedad relativa
            humidity_min =('relative_humidity_2m', 'min'),  # Mínimo de humedad relativa
            humidity_max =('relative_humidity_2m', 'max'),  # Máximo de humedad relativa
            wind_speed = ('wind_speed_10m', 'mean'),  # Promedio de velocidad del viento
            precipitation = ('precipitation', 'sum')  # Suma total de precipitaciones
            
        ).reset_index()
        daily_df.rename(columns={"Dia": "Fecha"}, inplace=True)
        daily_df.set_index(pd.to_datetime(daily_df['Fecha']), inplace=True)
        daily_df = daily_df.drop("Fecha", axis=1)
        out_df = daily_df.round(3)
    else:
        out_df = df

    # Guardamos en archivo con la día en la marca de tiempo
    filename = "data/data_om_"+id+"_"+datetime.now().strftime("%Y%m%d")+".csv"
    if agregar_diario:
        filename = "data/data_om_"+id+"_diario_"+datetime.now().strftime("%Y%m%d")+".csv"
    out_df.to_csv(filename)
    print("Datos Open-Meteo procesados y guardados.")
    return filename

# Aquí empieza la ejecución
agregar_diario = False
datos = obtener_datos_historicos_open_meteo(START_DATE, END_DATE, LAT, LON)
filename = procesar_datos_open_meteo(datos, "puerto_cruz", agregar_diario)
print(filename)


## **Script 2: Recopilación y Procesamiento de Datos de AEMET**
El objetivo de este script es recopilar datos históricos de  AEMET, procesarlos y guardarlos como archivos CSV para su posterior análisis.

### **Código del Script**

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import quote

# Configuración
DAYS = 7  # Últimos 15 días
END_DATE = datetime.now().date() - timedelta(days=0)
START_DATE = END_DATE - timedelta(days=DAYS)

AEMET_API_KEY = "TU_CLAVE_API" # Reemplaza con tu API Key de AEMET
AEMET_URL = "https://opendata.aemet.es/opendata/"

# ID de estación
ID = 'C459Z'

# Función para obtener datos históricos de AEMET
def obtener_datos_historicos_aemet(api_key, start_date, end_date, station_id):
    headers = {"api_key": api_key}
    fecha_ini = start_date.strftime('%Y-%m-%dT%H:%M:%SUTC')
    fecha_fin = end_date.strftime('%Y-%m-%dT%H:%M:%SUTC')
    url = f"{AEMET_URL}api/valores/climatologicos/diarios/datos/fechaini/{fecha_ini}/fechafin/{fecha_fin}/estacion/{station_id}"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200 and 'datos' in response.json():
        data_url = response.json()["datos"]
        data_response = requests.get(data_url)
        if data_response.status_code == 200:
            return data_response.json()
        else:
            raise Exception(f"Error al descargar datos de AEMET: {data_response.status_code}")
    else:
        raise Exception(f"Error en la consulta a AEMET: {response.status_code}")

def procesar_datos_aemet(data, id):
    df = pd.DataFrame(data)
    df['Fecha'] = pd.to_datetime(df['fecha'])
    df.set_index("Fecha", inplace=True)
    df = df[['tmed', 'tmin', 'tmax', 'hrMedia', 'hrMax', 'hrMin', 'velmedia', 'prec']]
    df = df.rename(columns={
        "tmed": "temperature",
        "tmin": "temperature_min",
        "tmax": "temperature_max",
        "velmedia": "wind_speed",
        "hrMedia": "humidity",
        "hrMin": "humidity_min",
        "hrMax": "humidity_max",
        "prec": "precipitation",
    })

    # Reemplazar comas por puntos y convertir a numérico
    df = df.map(lambda x: float(x.replace(',', '.')) if isinstance(x, str) else x)
    
    filename = "data/data_aemet_"+id+"_"+datetime.now().strftime("%Y%m%d")+".csv"
    df.to_csv(filename)
    print("Datos AEMET procesados y guardados.")
    return filename
    
# Aquí empieza la ejecución
datos = obtener_datos_historicos_aemet(AEMET_API_KEY, START_DATE, END_DATE, ID)
filename = procesar_datos_aemet(datos, ID)
print(filename)

## Automatización de scripts

### **Automatización con Crontab y Ficheros de Log**
En Linux el sistema estándar de programación de scripts es mediante el servicio `cron`. Se basa en el uso de un archivo llamado `crontab` (abreviatura de "cron table") donde se definen las tareas y su programación. Estas tareas pueden ser scripts, comandos o programas que se ejecutan en momentos específicos, según el cronograma definido.

#### Formato de crontab
```bash
*  *  *  *  * comando
-  -  -  -  -
|  |  |  |  |
|  |  |  |  +--- Día de la semana (0 - 7) (Domingo = 0 o 7)
|  |  |  +------ Mes (1 - 12)
|  |  +--------- Día del mes (1 - 31)
|  +------------ Hora (0 - 23)
+--------------- Minuto (0 - 59)
```
#### Instrucciones

1. **Editar Crontab**:
   Ejecuta el siguiente comando para abrir el editor de crontab:
   ```bash
   crontab -e
   ```

2. **Añadir las tareas programadas**:
   Añade las líneas siguientes al archivo crontab para programar la ejecución diaria de los scripts y registrar sus salidas en un fichero de log:

   ```bash
   0 6 * * * python3 /ruta/al/script/recopilacion_datos_om.py >> /ruta/al/logs/recopilacion_om.log 2>&1
   0 7 * * * python3 /ruta/al/script/recopilacion_datos_aemet.py >> /ruta/al/logs/recopilacion_aemet.log 2>&1
   ```

   - **`>>`**: Agrega la salida al fichero de log especificado.
   - **`2>&1`**: Redirige tanto la salida estándar como los errores al fichero.

3. **Verificar las entradas en Crontab**:
   Confirma que las tareas se han añadido correctamente ejecutando:
   ```bash
   crontab -l
   ```

4. **Validar el log**:
   - Ejecuta el script manualmente para comprobar que el log funciona:
     ```bash
     python3 /ruta/al/script/recopilacion_datos_om.py >> /ruta/al/logs/recopilacion_om.log 2>&1
     ```
   - Abre el fichero de log:
     ```bash
     cat /ruta/al/logs/recopilacion_om.log
     ```

---

### **Automatización con Windows Task Scheduler**
**Windows Task Scheduler** es una herramienta integrada en los sistemas operativos Windows que permite automatizar tareas repetitivas en un equipo. Permite programar la ejecución de aplicaciones, scripts, y comandos en horarios específicos o cuando ocurren ciertos eventos, como el inicio del sistema o el inicio de sesión de un usuario. Con una interfaz gráfica intuitiva, Task Scheduler facilita la creación de tareas mediante asistentes que permiten definir los desencadenantes (horarios o eventos), las acciones a ejecutar (como abrir un programa o ejecutar un script), y las condiciones adicionales que deben cumplirse para ejecutar la tarea.

#### Instrucciones

1. **Abrir el Programador de Tareas**:
   - Pulsa `Win + S` y busca "Programador de tareas".
   - Haz clic en **Crear tarea básica**.

2. **Configurar la Tarea Básica**:
   - **Nombre**: Introduce un nombre como "Recopilación de Datos OM".
   - **Descripción**: Opcionalmente, añade una breve descripción.

3. **Configurar el Disparador**:
   - Selecciona la opción "Diariamente" y establece la hora de ejecución.

4. **Configurar la Acción**:
   - Selecciona "Iniciar un programa".
   - Busca el ejecutable de Python (`python.exe` o `python3.exe`).
   - Introduce la ruta del script en el campo "Agregar argumentos", por ejemplo:
     ```plaintext
     /ruta/al/script/recopilacion_datos_om.py
     ```

5. **Configurar Logs (opcional)**:
   - En el campo "Agregar argumentos", redirige la salida a un fichero de log:
     ```plaintext
     /ruta/al/script/recopilacion_datos_om.py >> C:\ruta\logs\recopilacion_om.log 2>&1
     ```
6. **Revisar y Guardar**:
   - Revisa la configuración y haz clic en **Finalizar**.

7. **Repetir para el segundo script**:
   - Sigue el mismo proceso para el script `recopilacion_datos_aemet.py`.



---
